In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
#pd.set_option('display.max_columns', 200)

# 1. Load the dataset
df = pd.read_csv("uber.csv")
print("Original Dataset Shape:", df.shape)


In [ ]:
# 2. Initial Inspection
print(df.info())
print(df.describe())
print("Missing values:\n", df.isnull().sum())


In [ ]:
# 3. Data Cleaning
# Drop rows with any missing values
df.dropna(inplace=True)

# Remove rows with invalid fare amounts
df = df[(df['fare_amount'] > 0) & (df['fare_amount'] < 1000)]

# Remove rows with invalid passenger counts (1–6 considered valid)
df = df[(df['passenger_count'] > 0) & (df['passenger_count'] <= 6)]

# Convert datetime
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'], errors='coerce')
df.dropna(subset=['pickup_datetime'], inplace=True)

# Optional: Drop longitude/latitude rows outside NYC area (basic bounding box)
df = df[
    (df['pickup_longitude'] > -75) & (df['pickup_longitude'] < -72) &
    (df['pickup_latitude'] > 40) & (df['pickup_latitude'] < 42)
]



In [ ]:
# 4. Feature Engineering
df['hour'] = df['pickup_datetime'].dt.hour
df['day'] = df['pickup_datetime'].dt.day
df['month'] = df['pickup_datetime'].dt.month
df['weekday'] = df['pickup_datetime'].dt.day_name()

# Peak hours flag
df['peak_hour'] = df['hour'].apply(lambda x: 'Peak' if x in [7,8,9,17,18,19] else 'Off-Peak')



In [ ]:
# 5. Distance Calculation (if dropoff columns exist)
if {'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'}.issubset(df.columns):
    from geopy.distance import geodesic

    def is_valid_coord(lat, lon):
        return -90 <= lat <= 90 and -180 <= lon <= 180

    def calculate_distance(row):
        try:
            pickup_lat = row['pickup_latitude']
            pickup_lon = row['pickup_longitude']
            dropoff_lat = row['dropoff_latitude']
            dropoff_lon = row['dropoff_longitude']
            
            if is_valid_coord(pickup_lat, pickup_lon) and is_valid_coord(dropoff_lat, dropoff_lon):
                pickup = (pickup_lat, pickup_lon)
                dropoff = (dropoff_lat, dropoff_lon)
                return geodesic(pickup, dropoff).km
            else:
                return np.nan
        except:
            return np.nan

    # Apply only if dropoff columns exist
    if {'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'}.issubset(df.columns):
        df['distance_km'] = df.apply(calculate_distance, axis=1)
        df.dropna(subset=['distance_km'], inplace=True)
        df = df[df['distance_km'] > 0]



In [ ]:
# 6. Descriptive Statistics & Plots
print("\nFare Amount Stats:")
#print(df['fare_amount'].describe())

# 1. Central Tendency & Spread
fare_stats = {
    'Mean': df['fare_amount'].mean(),
    'Median': df['fare_amount'].median(),
    'Mode': df['fare_amount'].mode()[0],
    'Standard Deviation': df['fare_amount'].std()
}
print("🔹 Fare Amount - Central Tendency and Spread:")
for key, value in fare_stats.items():
    print(f"{key}: ${value:.2f}")

# 2. Quartiles and IQR
Q1 = df['fare_amount'].quantile(0.25)
Q2 = df['fare_amount'].quantile(0.50)
Q3 = df['fare_amount'].quantile(0.75)
IQR = Q3 - Q1
min_val = df['fare_amount'].min()
max_val = df['fare_amount'].max()

print("\n🔹 Quartiles and Range:")
print(f"Q1 (25th percentile): ${Q1:.2f}")
print(f"Q2 (Median): ${Q2:.2f}")
print(f"Q3 (75th percentile): ${Q3:.2f}")
print(f"IQR (Q3 - Q1): ${IQR:.2f}")
print(f"Minimum Fare: ${min_val:.2f}")
print(f"Maximum Fare: ${max_val:.2f}")

# 3. Outlier Detection (Using IQR)
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
outliers = df[(df['fare_amount'] < lower_bound) | (df['fare_amount'] > upper_bound)]

print(f"\n🔹 Outlier Detection:")
print(f"Lower Bound: ${lower_bound:.2f}")
print(f"Upper Bound: ${upper_bound:.2f}")
print(f"Number of Outliers: {outliers.shape[0]}")
print(f"Outlier Fare Range: ${outliers['fare_amount'].min():.2f} to ${outliers['fare_amount'].max():.2f}")


# Histogram of Fare Amount
plt.figure(figsize=(10, 5))
sns.histplot(df['fare_amount'], bins=50, kde=True, color='skyblue')
plt.title('Fare Amount Distribution')
plt.xlabel('Fare Amount ($)')
plt.ylabel('Frequency')
plt.grid(True)
plt.tight_layout()
plt.savefig("fare_histogram.png")
plt.show()

# Boxplot of Fare Amount
plt.figure(figsize=(8, 3))
sns.boxplot(x=df['fare_amount'], color='salmon')
plt.title('Fare Amount Boxplot')
plt.xlabel('Fare Amount ($)')
plt.grid(True)
plt.tight_layout()
plt.savefig("fare_boxplot.png")
plt.show()



In [ ]:
# Scatterplot: Fare vs Distance
plt.figure(figsize=(10, 5))
sns.scatterplot(x='distance_km', y='fare_amount', data=df, alpha=0.5)
plt.title('Fare Amount vs. Distance Traveled')
plt.xlabel('Distance (km)')
plt.ylabel('Fare Amount ($)')
plt.grid(True)
plt.tight_layout()
plt.savefig("fare_vs_distance.png")
plt.show()

# Boxplot: Fare by Hour
plt.figure(figsize=(12, 5))
sns.boxplot(x='hour', y='fare_amount', data=df)
plt.title('Fare Amount vs. Hour of Day')
plt.xlabel('Hour of Day')
plt.ylabel('Fare Amount ($)')
plt.grid(True)
plt.tight_layout()
plt.savefig("fare_vs_hour.png")
plt.show()

# Correlation matrix
numerical_cols = ['fare_amount', 'passenger_count', 'distance_km', 'hour', 'day', 'month']
corr_matrix = df[numerical_cols].corr()

plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix of Key Variables')
plt.tight_layout()
plt.savefig("correlation_matrix.png")
plt.show()

# Correlation matrix
numerical_cols = ['fare_amount', 'passenger_count', 'distance_km', 'hour', 'day', 'month']
corr_matrix = df[numerical_cols].corr()

plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix of Key Variables')
plt.tight_layout()
plt.savefig("correlation_matrix.png")
plt.show()


In [ ]:
# Ensure pickup_datetime is datetime
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])

# Extract time-based features
df['hour'] = df['pickup_datetime'].dt.hour
df['day'] = df['pickup_datetime'].dt.day
df['month'] = df['pickup_datetime'].dt.month
df['weekday'] = df['pickup_datetime'].dt.day_name()

# Peak/Off-Peak Labeling
def label_peak(hour):
    if 7 <= hour <= 9 or 17 <= hour <= 19:
        return 'Peak'
    return 'Off-Peak'

df['peak_offpeak'] = df['hour'].apply(label_peak)

# Convert categorical variables into category type (Power BI can read this too)
df['weekday'] = df['weekday'].astype('category')
df['peak_offpeak'] = df['peak_offpeak'].astype('category')


In [ ]:
# -----------------------------
# 7. Save Cleaned & Enhanced Dataset
# -----------------------------
df.to_csv("uber_enhanced.csv", index=False)
print("\nCleaned and enhanced dataset saved as 'uber_enhanced.csv'")
